# GitHub Group Creation

> Create GitHub group based on the provided group information

In [ ]:
#| default_exp gh_group

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export
from github import Github
import github
import json

In [ ]:
#| export
class GitHubGroup:
    def __init__(self,
                 credentials_fp="", # the file path to the credential json
                 org="", # the organization name
                ):
        self.github = None
        self.org = None
        
        if credentials_fp != "":
            self.auth_github(credentials_fp)
        if org != "":
            self.set_org(org)

    def auth_github(self,
                    credentials_fp: str # the personal access token generated at GitHub Settings
                   ):
        "Authenticate GitHub account with the provided credentials"
        with open(credentials_fp, "r") as f:
            token = json.load(f)["GitHub Token"]
        self.github = Github(token)
        # check authorization
        _ = self.github.get_user().get_repos()[0]
        
    def set_org(self,
                org: str # the target organization name
               ):
        "Set the target organization for repo creation"
        self.org = self.github.get_organization(org)

    def create_repo(self,
                    repo_name: str, # repository name
                    repo_template="", # template repository that new repo will use. If empty string, an empty repo will be created. Put in the format of "<owner>/<repo>"
                    private=True, # visibility of the created repository
                    description="", # description for the GitHub repository
                    personal_account=False, # create repos in personal GitHub account
                    ) -> github.Repository.Repository:
        "Create a repository, either blank, or from a template"
        if self.org is None and personal_account:
            raise ValueError("Organization is not set")
        if personal_account:
            parent = self.github.get_user()
        else:
            parent = self.org
        if repo_template == "":
            return parent.create_repo(
                name=repo_name,
                private=private,
                description=description
            )
        # create from template
        return parent.create_repo_from_template(
            name=repo_name,
            repo=self.get_repo(repo_template),
            private=private,
            description=description,
        )
    
    def get_repo(self,
                 repo_full_name: str # full name of the target repository
                ) -> github.Repository.Repository:
        "To get a repository by its name"
        return self.github.get_repo(repo_full_name)
    
    def rename_files(self,
                     repo: github.Repository.Repository, # the repository that we want to rename file
                     og_filename: str, # old file name
                     new_filename: str # new file name
                    ):
        "Rename the file by delete the old file and commit the new file"
        file = repo.get_contents(og_filename)
        repo.create_file(new_filename, "rename files", file.decoded_content)
        repo.delete_file(og_filename, "delete old files", file.sha)
        
    def add_collaborators(self,
                          repo: github.Repository.Repository, # target repository
                          collaborator:str, # GitHub username of the collaborator
                          permission:str # `pull`, `push` or `admin`
                         ):
        "add collaborators to the repository"
        repo.add_to_collaborators(collaborator, permission)
        
    def create_group_repo(self,
                          repo_name: str, # group repository name
                          collaborators: [str], # list of collaborators GitHub id
                          permission: str, # the permission of collaborators
                          rename_files=dict(), # dictionary of files renames {<og_name>:<new_name>}
                          repo_template="", # If empty string, an empty repo will be created. Put in the format of "<owner>/<repo>"
                          private=True, # visibility of the created repository
                          description="", # description for the GitHub repository
                         ) -> github.Repository.Repository:
        "Create a Group Repository"
        repo = self.create_repo(repo_name, repo_template, private, description)
        for og_name, new_name in rename_files.values():
            self.rename_files(repo, og_name, new_name)
        for collaborator in collaborators:
            self.add_collaborators(repo, collaborator)
        return repo

## GitHub Authentication

View this [document](https://docs.google.com/document/d/1RvZnOX6nh0bXn6Zh4U-Yxazukuez5oGrtcP8mN-Roco/edit?usp=sharing) for how to set up your GitHub Personal Access Token. (TODO: be sure to specify scopes)

In [ ]:
credentials_fp = "credentials.json"
credentials_fp = "../../credentials.json" #| hide_line
g = GitHubGroup(credentials_fp="../../credentials.json", org="COGS118A")

Optionally, you can instansiate a GitHubGroup object and authenticate yourself by calling the following method.

In [ ]:
show_doc(GitHubGroup.auth_github)

---

### GitHubGroup.auth_github

>      GitHubGroup.auth_github (credentials_fp:str)

Authenticate GitHub account with the provided credentials

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| credentials_fp | str | the personal access token generated at GitHub Settings |

In [ ]:
g = GitHubGroup()
g.auth_github("../../credentials.json")
g.github.get_user().name

'Scott Yang'

## GitHub Organization Settings

Usually, you want to create students repositories under a course GitHub organization. To set the target organization, you can call the following function.

In [ ]:
show_doc(GitHubGroup.set_org)

---

### GitHubGroup.set_org

>      GitHubGroup.set_org (org:str)

Set the target organization for repo creation

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| org | str | the target organization name |

In [ ]:
g.set_org("COGS118A")
g.org

Organization(login="COGS118A")

## Create GitHub Group in one Call

In [ ]:
show_doc(GitHubGroup.create_group_repo)

---

### GitHubGroup.create_group_repo

>      GitHubGroup.create_group_repo (repo_name:str,
>                                     collaborators:[<class'str'>],
>                                     permission:str, rename_files={},
>                                     repo_template='', private=True,
>                                     description='')

Create a Group Repository

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| repo_name | str |  | group repository name |
| collaborators | [<class 'str'>] |  | list of collaborators GitHub id |
| permission | str |  | the permission of collaborators |
| rename_files | dict | {} | dictionary of files renames {<og_name>:<new_name>} |
| repo_template | str |  | If empty string, an empty repo will be created. Put in the format of "<owner>/<repo>" |
| private | bool | True | visibility of the created repository |
| description | str |  | description for the GitHub repository |
| **Returns** | **Repository** |  |  |

In [ ]:
g.create_group_repo

<bound method GitHubGroup.create_group_repo of <__main__.GitHubGroup object>>

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()